<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#To-load-the-data" data-toc-modified-id="To-load-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>To load the data</a></span></li><li><span><a href="#Load-the-strong-annotation-and-the-ground-thruth" data-toc-modified-id="Load-the-strong-annotation-and-the-ground-thruth-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the strong annotation and the ground thruth</a></span></li><li><span><a href="#Optimization" data-toc-modified-id="Optimization-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Optimization</a></span></li></ul></div>

# Import

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import time
import os
import matplotlib.pyplot as plt
import dcase_util as dcu
from evaluation_measures import event_based_evaluation  # evaluation system from DCASE

from Encoder import Encoder

# To load the data

In [2]:
import numpy as np
import os

class_correspondance = {"Alarm_bell_ringing": 0, "Speech": 1, "Dog": 2, "Cat": 3, "Vacuum_cleaner": 4,
                        "Dishes": 5, "Frying": 6, "Electric_shaver_toothbrush": 7, "Blender": 8, "Running_water": 9}


def load_mil():
    strong_prediction = np.load(os.path.join("data", "mil_strong_prediction.pkl"))
    tmp = dict()

    # Fill the missing curve with zero
    for filename in strong_prediction:
        new_data = np.zeros((431, 10))

        for cls in strong_prediction[filename]:
            new_data[:, class_correspondance[cls]] = strong_prediction[filename][cls]

        tmp[filename] = new_data.copy()

    # Create a list of filename in the same order than the dict
    filename_list = ["%s.wav" % k for k in tmp.keys()]

    # Create a list
    output_data = np.array(list(tmp.values()))

    return output_data, filename_list


def load_baseline():
    filenames = np.load(os.path.join("data", "X_test_fn.npy"))
    strong_prediction = np.load(os.path.join("data", "baseline_strong_prediction.npy"))

    return strong_prediction, filenames


def load_ground_truth():
    test_unique_names = np.load(os.path.join("data", "X_test_fn.npy"))

    with open(os.path.join("data", "test.csv"), "r") as f:
        test_metadata = f.read().splitlines()[1:]
    test_metadata = np.array([info.split("\t") for info in test_metadata])
    test_medata_filenames = np.unique(test_metadata[:, 0])

    y_test = []
    for name in test_medata_filenames:
        # remove  missing files
        if name not in test_unique_names:
            continue

        # create a tag level prediction (aka weak labels)
        output = [0] * 10

        for d in test_metadata:
            if d[0] == name:
                output[class_correspondance[d[-1]]] = 1
        y_test.append(output)
    y_test = np.array(y_test)

    return y_test


# Load the strong annotation and the ground thruth

In [3]:
# strong_annotation, filenames = Loader.load_mil()
strong_prediction, filenames = load_baseline()

# ground truth
with open(os.path.join("data", "test.csv"), "r") as f:
    y_true = f.read().splitlines()[1:]

# Optimization

In [4]:
from Encoder import Encoder
import optimizers.DichotomicOptimizer as Optimizer

In [5]:
class_list = ['Alarm_bell_ringing', 'Speech', 'Dog', 'Cat', 'Vacuum_cleaner', 'Dishes', 'Frying', 'Electric_shaver_toothbrush', 'Blender', 'Running_water']

# Create the encoder that will be used
encoder = Encoder(
    classes=class_list,
    temporal_precision = 200,  # ms
    clip_length = 10,          # s
    minimal_segment_step = 200 # ms
)

print(encoder)

# Create the optimizer
optimizer = Optimizer.ThresholdOptimizer(
    {
        "threshold": (0.1, 0.9)
    },
    
    encoder = encoder,
    step = 3,
    nb_recurse = 2,
    nb_digit = 4,
    nb_process = 1
)

# Perform the optimization
optimizer.fit(y_true, strong_prediction, filenames)

Evaluate :  {'threshold': 0.1}
Evaluate :  {'threshold': 0.5}
Evaluate :  {'threshold': 0.9}


IndexError: list index out of range